

This coding script makes EPE (each pixel value evaluation) Analysis which explained in the article.


----------

CONTENTS

-    [1. importing necessary libraries](#1)    
         1.1 importing puclic libraries:
               --if this box is giving an error, this mean you need to check libraries in this cell, 
               you need    to install them               
         1.2 importing user-defined libraries 
               --if this box is giving an error, this mean you need to check
               ipynb files(jupyter notebook) with the name of  'project_helper_functions_classes' and 
               'Feature_extraction_classes_functions'
               within your working directory
                              
-    [2. Preperation of input informations](#2) 
         -- check and set parts    
         2.1  Specify paths                       
         2.2 Specify analysis features
         2.3 preparing file paths and creating necessary folders
                            
-    [3. EPE Analysis](#3)    
         3.1 Data Preparation
         3.2  EPE analysis for each MRI type
         3.3  EPE analysis for sequences (Collective Analysis)
         3.4 unite all acceptable features
         

<a name='1'></a>
# 1. importing necessary libraries

### 1.1 importing puclic libraries

In [1]:
import pandas as pd
import os
import pickle

### 1.2 importing user-defined libraries 

In [2]:
from ipynb.fs.full.project_helper_functions_classes import *
from ipynb.fs.full.Feature_extraction_classes_functions import *

# 2. Preperation of input informations

### 2.1 Specify paths

In [3]:
data_root_folder = 'data'
output_root_folder = 'output' 
MRC_stat_results_folder = 'MRC_stat_results'
EPE_stat_results_folder = 'EPE_stat_results'  # folder name for saving statistic results, will be created in output folder
feature_record_folder = 'features'
case_info_excel_file = 'MRI_informations.xlsx'
# spesify csv files for loading final selected features 
features_csv = 'selected_features.csv'
features_collective_csv = 'selected_features_coll.csv' # if you made collective analysis before
# specify csv files for saving EPE analysis specificity-sensitivity stat results:
EPE_stat_file_name = 'stat_results.csv'
coll_EPE_stat_file_name = 'coll_stat_results.csv'
all_selected_EPE_stat_file_name = 'EPE_selected_features.csv'

### 2.2 Specify analysis features

In [4]:
# specify balance accuracy limit for EPE analysis
specificity_min = 0.90
sensitivity_min = 0.60

## specify maximum number for storage values per case for protecting memory
## PCVL : per case value limit 
PCVL = 1000

# specify if you would like to proccess collective analysis by evaluating same sequences together
make_collective_analysis = True

# specify MRI sequences with their subfolders if you would like to make collective analysis (if : make_collective_analysis = True)
sequences = {'t1': ['t1_tra', 't1_sag'], 't2' : ['t2_cor', 't2_tra']}

# specify if you would like to store features' values
record_features = True   # True or False
follow_proccess = 'only_stats'   ## 'only_stats, or 'all', or False

### 2.3 preparing file paths and creating necessary folders

In [5]:
stripped_path =  os.path.abspath(os.path.join(data_root_folder, 'stripped_data'))
original_path =  os.path.abspath(os.path.join(data_root_folder, 'ordered_data'))
output_path = os.path.abspath(output_root_folder)
features_path = os.path.join(output_path, MRC_stat_results_folder, features_csv)
EPE_stat_results_folder_path = create_folder(os.path.join(output_path, EPE_stat_results_folder))
feature_record_folder_path = create_folder(os.path.join(output_path, feature_record_folder))
case_info_path = os.path.abspath(os.path.join(output_root_folder, case_info_excel_file))

if make_collective_analysis:
    collective_features_path = os.path.join(output_path, MRC_stat_results_folder, features_collective_csv)

## specify main folders lists
situations = take_folder_list(stripped_path)

## specify controls and patients folders' names    
control_id = [situation for situation in situations if 'c' == situation[0]][0]
patient_id = [situation for situation in situations if 'p' == situation[0]][0]

## create folder for saving statistic results
stat_result_path = create_folder(os.path.join(output_root_folder, EPE_stat_results_folder))

# 3. EPE Analaysis

### 3.1 Data Preparation

In [6]:
# 3.1.1 import features' informations
features_infos = pd.read_csv(features_path)

if make_collective_analysis:
    features_coll_infos = pd.read_csv(collective_features_path)
    
# 3.1.2 specify file lists for proccess 
file_paths, file_paths_coll = {}, {}
file_list, folder_path_list = take_special_file_list(situations, stripped_path, original_path)

## recording file paths based on single folders clinical conditions(situation):
for folder in features_infos['MRI_type'].unique():
    file_paths[folder] = special_filter_path(file_list, situations, [folder])
        
## recording file paths for united folders based on each sequence folders and clinical conditions(situation):
if make_collective_analysis:
    for sq in features_coll_infos['sequence'].unique():
        file_paths_coll[sq] = special_filter_path(file_list, situations, sequences[sq])
    

### 3.2  EPE analysis for each MRI type

In [7]:
def EPE_analysis_(key):

    ## specify features belong to specific folder (key)
    features = features_infos[features_infos['MRI_type'] == key]
    EPE = EPE_analysis(control_id, patient_id, PCVL  = PCVL, 
                       feature_output_folder = feature_record_folder_path,
                       follow_proccess = follow_proccess)
    EPE_df = EPE.activate_EPE_analysis(features, 
                                       file_paths[key][control_id],  file_paths[key][patient_id],
                                       specificity_min = specificity_min, sensitivity_min = sensitivity_min)    
  
    ## save results to csv file 
    result_output_path =  os.path.join(EPE_stat_results_folder_path, key + '-' + EPE_stat_file_name)
    EPE_df.sort_values(by = 'specificity(EPE)', ascending= False, inplace=True)
    EPE_df.to_csv(result_output_path, index =False) 



In [8]:
if __name__ == '__main__':
    
    run_apply_async_multiprocessing(EPE_analysis_, file_paths.keys(), 4)

  0%|                                                     | 0/5 [00:00<?, ?it/s]

scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
 specificity score: 0.9310344827586207, sensitivity score: 0.6060606060606061
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not fou

scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were

scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits


100%|████████████████████████████████████████████| 5/5 [10:16<00:00, 123.35s/it]


### 3.3  collective EPE analysis for each sequence

In [9]:
df_stat = pd.DataFrame()

def collective_EPE_analysis_(key):
    ## specify features belong to specific folder (key)
    features = features_coll_infos[features_coll_infos['sequence'] == key]
    EPE = EPE_analysis(control_id, patient_id, PCVL  = PCVL, 
                       feature_output_folder = feature_record_folder_path,
                       follow_proccess = follow_proccess)
    EPE_df = EPE.activate_EPE_analysis(features, 
                                       file_paths_coll[key][control_id],  file_paths_coll[key][patient_id],
                                       specificity_min = specificity_min, sensitivity_min = sensitivity_min,
                                      activate_collective_analysis = True, 
                                    info_path = case_info_path, 
                                       sequence_folders = sequences[key])   

     ## save results to csv file 
        
    result_output_path =  os.path.join(EPE_stat_results_folder_path, key + '-' + coll_EPE_stat_file_name)
    EPE_df.sort_values(by = 'specificity(EPE)', ascending= False, inplace=True)
    EPE_df.to_csv(result_output_path, index =False) 




In [10]:
if __name__ == '__main__':
    
    run_apply_async_multiprocessing(collective_EPE_analysis_, file_paths_coll.keys(), 3)

  0%|                                                     | 0/2 [00:00<?, ?it/s]

scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
 specificity score: 0.9090909090909091, sensitivity score: 0.6
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desire

scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were

scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were not found above desired limits
scores were

100%|███████████████████████████████████████████| 2/2 [49:11<00:00, 1475.95s/it]


### 3.4 unite all acceptable features

In [11]:
all_results_df = pd.DataFrame()

## get MRI type reuslts
for key in file_paths.keys():
    result_output_path =  os.path.join(EPE_stat_results_folder_path, key + '-' + EPE_stat_file_name)
    df = pd.read_csv(result_output_path)
    all_results_df = pd.concat([df, all_results_df])
    
## get sequence (collective) results
if make_collective_analysis:
    for key in file_paths_coll.keys():
        result_output_path =  os.path.join(EPE_stat_results_folder_path, key + '-' + coll_EPE_stat_file_name)        
        df = pd.read_csv(result_output_path)
        df.rename(columns = {'sequence':'MRI_type'}, inplace = True)
        all_results_df = pd.concat([df, all_results_df])
        
## save data
all_results_df.sort_values(by = 'specificity(EPE)', ascending=False, inplace=True)
result_output_path = os.path.join(output_root_folder, EPE_stat_results_folder, all_selected_EPE_stat_file_name)
all_results_df.to_csv(result_output_path, index =False) 